In [ ]:
#importy
from datetime import datetime
import numpy as np

import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from nltk.sentiment import SentimentIntensityAnalyzer
from tqdm import tqdm
import json


from tsplot import tsplot
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import arma_order_select_ic
import matplotlib.pylab as plt
import warnings
warnings.simplefilter(action='ignore', category=(FutureWarning, DeprecationWarning))

# Cíle projektu
Cílem projektu je analyzovat sentiment získaný ze zpráv z komunikační aplikace Slack. 
# Data
Jako dataset jsem použil zprávy ze Slacku: [zdroj]. Nejdříve jsem musel data očistit a provést samotnou analýzu sentimentu, pomocí natrénovaného analyzátoru z balíčku nltk.
Analýza vrací 4 složky/metriky: positive/negative/neutral a compound, které všechny budeme analyzovat.  

In [ ]:
#DATA PREPARATION (Takes about 10 minutes, so I converted final dataset to csv slack_data_pol.csv
sia = SentimentIntensityAnalyzer()
df = pd.read_csv('slack_data.csv', sep=',')
df = df[['sent', 'text']]
df['text'] = df['text'].astype(str)

df['sent'] = df['sent'].apply(lambda x: datetime.strptime(x[:10], '%Y-%m-%d'))
df['pol'] = df['text'].apply(lambda x: sia.polarity_scores(x))
df.to_csv('slack_data_pol_full.csv')
df['CompoundPol'] = df['pol'].apply(lambda x: x['compound'])
df = df[['sent', 'CompoundPol']]
df = df.groupby(['sent']).mean()
df.to_csv('slack_data_pol.csv')

## Prvotní analýza
Na grafu dole můžeme vidět vizualizaci naší datové řady. Řada vznikla zprůměrováním určité metriky v rámci jednoho dne. Na grafech s různými počty analyzovaných lagů, můžeme vidět, že ACF začíná blízko k 1, ale pomalu klesá. PACF je významně vysoká mezi lagy 1-4, což znamená, že nálada konkrétní den, je přímo ovlivněna historií chatu zhruba 4 dny nazpět. Z grafu také vyčteme neexistenci sezónosti v rámci dní.  

In [ ]:
df = pd.read_csv('slack_data_pol.csv', sep=',', index_col=0, parse_dates=True, infer_datetime_format=True)
display(df.head())
display(df.tail(5))
df['CompoundPol'] = df['CompoundPol'].astype(float)
df = df['CompoundPol'].last('2Y')
df = df.asfreq('D', 'bfill')
tsplot(df, lags=30)

In [ ]:
tsplot(df, lags=353)

# Analýza po týdnech 

In [ ]:
df = pd.read_csv('slack_data_pol.csv', sep=',', index_col=0, parse_dates=True, infer_datetime_format=True)
df['CompoundPol'] = df['CompoundPol'].astype(float)
df = df['CompoundPol'].last('2Y')
df = df.asfreq('W', 'bfill')
tsplot(df, lags=49)

# Model